In [1]:
! python -V

Python 3.6.9 :: Anaconda, Inc.


In [2]:
# ! pip install kgforge

Each section can be executed independently if the following one is executed before.

## Forge

In [3]:
from kgforge.core import KnowledgeGraphForge

In [4]:
forge = KnowledgeGraphForge.from_config("demo-forge.yml")

## Resource

In [5]:
from kgforge.core import Resource

In [6]:
jane = Resource(type="Person", name="Jane Doe")

In [7]:
jane.name

'Jane Doe'

In [8]:
jane.email = "jane.doe@epfl.ch"

In [9]:
jane.email

'jane.doe@epfl.ch'

In [10]:
association = Resource(type="Association", agent=jane)

In [11]:
association.agent.name

'Jane Doe'

In [12]:
association.agent

Resource(_last_action=None, _validated=False, _synchronized=False, _store_metadata=None, type='Person', email='jane.doe@epfl.ch', name='Jane Doe')

In [13]:
print(association.agent)

{
    type: Person
    email: jane.doe@epfl.ch
    name: Jane Doe
}


## Resources

In [14]:
from kgforge.core import Resource, Resources

In [15]:
jane = Resource(type="Person", name="Jane Doe")

In [16]:
john = Resource(type="Person", name="John Smith")

In [17]:
persons = Resources(jane, john)

In [18]:
len(persons)

2

In [19]:
persons[1].name

'John Smith'

In [20]:
persons

[Resource(_last_action=None, _validated=False, _synchronized=False, _store_metadata=None, type='Person', name='Jane Doe'),
 Resource(_last_action=None, _validated=False, _synchronized=False, _store_metadata=None, type='Person', name='John Smith')]

In [21]:
print(persons)

[
    {
        type: Person
        name: Jane Doe
    }
    {
        type: Person
        name: John Smith
    }
]


In [22]:
resources = [jane, john]

In [23]:
persons_too = Resources(resources)

In [24]:
len(persons_too)

2

## Dataset

In [25]:
from kgforge.core import Resource, Resources
from kgforge.specializations.resources import Dataset

**Dataset**

In [26]:
dataset = Dataset(forge, name="Interesting people")

In [27]:
type(dataset)

kgforge.specializations.resources.datasets.Dataset

In [28]:
isinstance(dataset, Resource) == True

True

In [29]:
print(dataset)

{
    type: Dataset
    name: Interesting people
}


**Parts**

In [30]:
jane = Resource(type="Person",
                name="Jane Doe",
                distribution=Resource(type="DataDownload", contentUrl="https://files.domain.ch/file_1.xyz"))

In [31]:
john = Resource(type="Person",
                name="John Smith",
                distribution=Resource(type="DataDownload", contentUrl="https://files.domain.ch/file_2.xyz"))

In [32]:
persons = Resources(jane, john)

In [33]:
forge.storing.register(persons)

<count> 2
<action> _register
<succeeded> True


In [34]:
dataset = Dataset(forge, name="Interesting people")

In [35]:
dataset.parts() == None

True

In [36]:
dataset.with_parts(persons)

<count> 2
<action> _freeze_one
<succeeded> True


In [37]:
print(dataset)

{
    type: Dataset
    hasPart:
    [
        {
            type: Person
            id: 0_version=1
            distribution:
            {
                contentUrl: https://files.domain.ch/file_1.xyz
            }
            name: Jane Doe
        }
        {
            type: Person
            id: 1_version=1
            distribution:
            {
                contentUrl: https://files.domain.ch/file_2.xyz
            }
            name: John Smith
        }
    ]
    name: Interesting people
}


In [38]:
parts = dataset.parts()

In [39]:
type(parts)

kgforge.core.resources.Resources

In [40]:
len(parts)

2

In [41]:
print(parts[0])

{
    type: Person
    id: 0_version=1
    distribution:
    {
        contentUrl: https://files.domain.ch/file_1.xyz
    }
    name: Jane Doe
}


In [42]:
files = dataset.files()

In [43]:
len(files)

2

In [44]:
print(files)

[
    {
        contentUrl: https://files.domain.ch/file_1.xyz
    }
    {
        contentUrl: https://files.domain.ch/file_2.xyz
    }
]


In [45]:
# FIXME Implement download() for DemoStore.
files.download("./downloaded/")

DemoStore is not supporting download()


**Files**

In [46]:
dataset = Dataset(forge, name="Interesting people")

In [47]:
dataset.files() == None

True

In [48]:
dataset.with_files("./data/")

In [49]:
dataset.files()

## Transforming

In [50]:
from kgforge.core import Resource, Resources
from kgforge.specializations.mappings import DictionaryMapping
from kgforge.specializations.mappers import DictionaryMapper

**Mapping**

In [51]:
scientists = [
    { "name": "Albert Einstein" },
    { "name": "Peter Higgs" },
]

In [52]:
DATA_SOURCE = "scientists-database"

In [53]:
forge.modeling.mappings(DATA_SOURCE)

{'Association': ['DictionaryMapping']}

In [54]:
mapping = forge.modeling.mapping("Association", DATA_SOURCE)

In [55]:
albert = forge.transforming.map(scientists[0], DictionaryMapper, mapping)

In [56]:
type(albert)

kgforge.core.resources.Resource

In [57]:
print(albert)

{
    type: Association
    id: https://knowledgegraph.domain.ch/associations/15
    agent:
    {
        type: Person
        gender:
        {
            type: LabeledOntologyEntity
            id: http://purl.obolibrary.org/obo/PATO_0000384
            label: male
        }
        name: Albert Einstein
    }
    distribution: LazyAction(operation=Store.upload, args=['./scientists-database/file.ext'])
}


In [58]:
scientists_mapped = forge.transforming.map(scientists, DictionaryMapper, mapping)

In [59]:
type(scientists_mapped)

kgforge.core.resources.Resources

In [60]:
len(scientists_mapped)

2

In [61]:
print(scientists_mapped[1])

{
    type: Association
    id: https://knowledgegraph.domain.ch/associations/11
    agent:
    {
        type: Person
        gender:
        {
            type: LabeledOntologyEntity
            id: http://purl.obolibrary.org/obo/PATO_0000384
            label: male
        }
        name: Peter Higgs
    }
    distribution: LazyAction(operation=Store.upload, args=['./scientists-database/file.ext'])
}


**Mapping - Creation**

In [62]:
forge.modeling.template("Association")

NB: DemoStore is not automatically including templates for nested typed resources yet
{
    type: Association
    agent:
    {
        type: Person
        name: hasattr
    }
}


In [63]:
mapping = DictionaryMapping("""
    type: Association
    id: forge.identifiers.format("associations", len(x.name))
    agent:
    {
        type: Person
        name: x.name
        gender: forge.ontologies.resolve("male", "sex")
    }
    distribution: forge.files.as_resource("./scientists-database/file.ext")
""")

**Mapping - Management**

In [64]:
path = "./data/mappings/scientists-database/DictionaryMapping/Association.hjson"

In [65]:
mapping.save(path)

In [66]:
mapping_loaded = DictionaryMapping.load(path)

In [67]:
print(mapping_loaded)

{
    type: Association
    id: forge.identifiers.format("associations", len(x.name))
    agent:
    {
        type: Person
        gender: forge.ontologies.resolve("male", "sex")
        name: x.name
    }
    distribution: forge.files.as_resource("./scientists-database/file.ext")
}


**Reshaping**

In [68]:
jane = Resource(type="Person", name="Jane Doe")

In [69]:
print(jane)

{
    type: Person
    name: Jane Doe
}


In [70]:
jane_reshaped = forge.transforming.reshape(jane, keep=["name"])

In [71]:
print(jane_reshaped)

{
    name: Jane Doe
}


In [72]:
john = Resource(type="Person", name="John Smith")

In [73]:
persons = Resources(jane, john)

In [74]:
print(persons)

[
    {
        type: Person
        name: Jane Doe
    }
    {
        type: Person
        name: John Smith
    }
]


In [75]:
persons_reshaped = forge.transforming.reshape(persons, keep=["name"])

In [76]:
print(persons_reshaped)

[
    {
        name: Jane Doe
    }
    {
        name: John Smith
    }
]


In [77]:
association = Resource(type="Association", agent=jane)

In [78]:
print(association)

{
    type: Association
    agent:
    {
        type: Person
        name: Jane Doe
    }
}


In [79]:
association_reshaped = forge.transforming.reshape(association, keep=["type", "agent.name"])

In [80]:
print(association_reshaped)

{
    type: Association
    agent:
    {
        name: Jane Doe
    }
}


**Reshaping - Versioning**

In [81]:
jane = Resource(type="Person", id="123456", name="Jane Doe")

In [82]:
forge.storing.register(jane)

<action> _register
<succeeded> True


In [83]:
print(jane)

{
    type: Person
    id: "123456"
    name: Jane Doe
}


In [84]:
jane_reshaped_freezed = forge.transforming.reshape(jane, keep=["type", "id"], versioned=True)

<action> _freeze_one
<succeeded> True


In [85]:
print(jane_reshaped_freezed)

{
    type: Person
    id: 123456_version=1
}


**Reshaping - Versioning - Error**

In [86]:
jane = Resource(type="Person", id="123456", name="Jane Doe")

In [87]:
john = Resource(type="Person", id="456789", name="John Smith")

In [88]:
persons = Resources(jane, john)

In [89]:
forge.transforming.reshape(persons, keep=["type", "id"], versioned=True)

<action> reshape
<error> resource not yet registered
<args> (<kgforge.core.transforming.reshaper.Reshaper object at 0x110c94940>, [Resource(_last_action=Action(operation='_freeze_one', succeeded=False, error='resource not yet registered'), _validated=False, _synchronized=False, _store_metadata=None, type='Person', id='123456', name='Jane Doe'), Resource(_last_action=None, _validated=False, _synchronized=False, _store_metadata=None, type='Person', id='456789', name='John Smith')], ['type', 'id'], True)


In [90]:
jane._last_action

Action(operation='_freeze_one', succeeded=False, error='resource not yet registered')

In [91]:
john._last_action == None

True

**Conversion - JSON**

In [92]:
# TODO Implement as_jsonld().

In [93]:
# jane = Resource(type="Person", name="Jane Doe")

In [94]:
# forge.storing.register(jane)

In [95]:
# forge.transforming.as_json(jane)

In [96]:
# forge.transforming.as_json(jane, expanded=True)

In [97]:
# forge.transforming.as_json(jane, store_metadata=True)

**Conversion - JSON-LD**

In [98]:
# TODO Implement as_jsonld().

In [99]:
# forge.transforming.as_jsonld(jane)

In [100]:
# forge.transforming.as_jsonld(jane, compacted=False)

In [101]:
# forge.transforming.as_jsonld(jane, store_metadata=True)

**Conversion - Triples**

In [102]:
# TODO Implement as_triples().

In [103]:
# forge.transforming.as_triples(jane)

In [104]:
# forge.transforming.as_triples(jane, store_metadata=True)

**Conversion - DataFrame**

In [105]:
# TODO Implement as_dataframe().

In [106]:
# forge.transforming.as_dataframe(jane)

In [107]:
# forge.transforming.as_dataframe(jane, store_metadata=True)

## Modeling

In [108]:
import json

In [109]:
from kgforge.core.resources import Resource, Resources

In [110]:
def pp(x): print(json.dumps(x, indent=4))

**Prefixes**

In [111]:
prefixes = forge.modeling.prefixes()

In [112]:
pp(prefixes)

{
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "schema": "http://schema.org/",
    "prov": "http://www.w3.org/ns/prov#"
}


**Types**

In [113]:
forge.modeling.types()

['Person', 'Association']

**Templates**

In [114]:
forge.modeling.template("Person")

{
    type: Person
    name: hasattr
}


In [115]:
forge.modeling.template("Association")

NB: DemoStore is not automatically including templates for nested typed resources yet
{
    type: Association
    agent:
    {
        type: Person
        name: hasattr
    }
}


In [116]:
# TODO Implement for DemoModel.
forge.modeling.template("Person", only_required=True)

DemoModel is not supporting template() with only_required=True
None


**Validation**

In [117]:
jane = Resource(type="Person", name="Jane Doe")

In [118]:
forge.modeling.validate(jane)

<action> _validate
<succeeded> True


In [119]:
jane._last_action

Action(operation='_validate', succeeded=True, error=None)

In [120]:
jane._validated == True

True

In [121]:
jane.email = "jane.doe@epfl.ch"

In [122]:
jane._validated == False

True

In [123]:
john = Resource(type="Person", name="John Smith")

In [124]:
persons = Resources(jane, john)

In [125]:
forge.modeling.validate(persons)

<count> 2
<action> _validate
<succeeded> True


In [126]:
john._validated

True

**Validation - Error**

In [127]:
mistake = Resource(type="Person")

In [128]:
forge.modeling.validate(mistake)

<action> _validate
<succeeded> False
<error> name is missing


In [129]:
mistake._last_action

Action(operation='_validate', succeeded=False, error='name is missing')

In [130]:
mistake._validated == False

True

In [131]:
nested_mistake = Resource(type="Association", agent=mistake)

In [132]:
forge.modeling.validate(nested_mistake)

<action> _validate
<succeeded> False
<error> name is missing


**Handlers - Identifiers**

In [133]:
forge.identifiers.format("persons", 753159)

'https://knowledgegraph.domain.ch/persons/753159'

**Handlers - Ontologies**

In [134]:
from kgforge.core.modeling.resolvers import ResolvingStrategy

In [135]:
female = forge.ontologies.resolve("female", "sex")

In [136]:
type(female)

kgforge.core.resources.Resource

In [137]:
print(female)

{
    type: LabeledOntologyEntity
    id: http://purl.obolibrary.org/obo/PATO_0000383
    label: female
}


In [138]:
print(forge.ontologies.resolve("male", "sex", strategy=ResolvingStrategy.ALL))

[
    {
        type: LabeledOntologyEntity
        id: http://purl.obolibrary.org/obo/PATO_0000384
        label: male
    }
    {
        type: LabeledOntologyEntity
        id: http://purl.obolibrary.org/obo/PATO_0000383
        label: female
    }
]


**Handlers - Files**

In [139]:
lazy_action = forge.files.as_resource("./data/mapping.hjson")

In [140]:
type(lazy_action)

kgforge.core.commons.actions.LazyAction

In [141]:
# FIXME Implement upload() for DemoStore.
lazy_action.execute()

DemoStore is not supporting _upload_one()


## Storing

In [142]:
from kgforge.core.resources import Resource, Resources

**Registration**

In [143]:
jane = Resource(type="Person", name="Jane Doe")

In [144]:
association = Resource(type="Association", agent=jane)

In [145]:
forge.storing.register(association)

<action> _register
<succeeded> True


In [146]:
association._store_metadata

{'version': 1, 'deprecated': False}

In [147]:
association._last_action

Action(operation='_register', succeeded=True, error=None)

In [148]:
association._synchronized == True

True

In [149]:
john = Resource(type="Person", name="John Smith")

In [150]:
association.agent = john

In [151]:
association._synchronized == False

True

**Registration - Error**

In [152]:
jane = Resource(type="Person", name="Jane Doe")

In [153]:
forge.storing.register(jane)

<action> _register
<succeeded> True


In [154]:
john = Resource(type="Person", name="John Smith")

In [155]:
persons = Resources(jane, john)

In [156]:
forge.storing.register(persons)

<count> 1
<action> _register
<succeeded> False
<error> resource already exists

<count> 1
<action> _register
<succeeded> True


In [157]:
jane._synchronized == False

True

In [158]:
john._synchronized == True

True

**Update**

In [159]:
jane = Resource(type="Person", name="Jane Doe")

In [160]:
association = Resource(type="Association", agent=jane)

In [161]:
forge.storing.register(association)

<action> _register
<succeeded> True


In [162]:
association._synchronized == True

True

In [163]:
association._store_metadata.version == 1

True

In [164]:
john = Resource(type="Person", name="John Smith")

In [165]:
association.agent = john

In [166]:
forge.storing.update(association)

<action> _register
<succeeded> True


In [167]:
association._synchronized == True

True

In [168]:
association._store_metadata.version == 2

True

**Tagging**

In [169]:
jane = Resource(type="Person", name="Jane Doe")

In [170]:
forge.storing.register(jane)

<action> _register
<succeeded> True


In [171]:
jane._store_metadata.version

1

In [172]:
jane._synchronized == True

True

In [173]:
forge.storing.tag(jane, "v1")

<action> _tag_one
<succeeded> True


**Tagging - Error**

In [174]:
john = Resource(type="Person", name="John Smith")

In [175]:
persons = Resources(jane, john)

In [176]:
john._synchronized == False

True

In [177]:
forge.storing.tag(persons, "v1")

<count> 1
<action> _tag_one
<succeeded> False
<error> resource version already tagged

<count> 1
<action> _tag_one
<succeeded> False
<error> resource not synchronized


**Deprecation**

In [178]:
jane = Resource(type="Person", name="Jane Doe")

In [179]:
forge.storing.register(jane)

<action> _register
<succeeded> True


In [180]:
jane._store_metadata.deprecated == False

True

In [181]:
forge.storing.deprecate(jane)

<action> _deprecate_one
<succeeded> True


In [182]:
jane._store_metadata.deprecated == True

True

## Querying

In [183]:
from kgforge.core.resources import Resource, Resources

**Retrieval**

In [184]:
jane = Resource(type="Person", name="Jane Doe")

In [185]:
forge.storing.register(jane)

<action> _register
<succeeded> True


In [186]:
forge.storing.tag(jane, "v1")

<action> _tag_one
<succeeded> True


In [187]:
jane.email = "jane.doe@epfl.ch"

In [188]:
forge.storing.update(jane)

<action> _register
<succeeded> True


In [189]:
jane_vlatest = forge.querying.retrieve(jane.id)

In [190]:
jane_vlatest._store_metadata.version == 2

True

In [191]:
jane_v1 = forge.querying.retrieve(jane.id, version=1)

In [192]:
jane_v1._store_metadata.version == 1

True

In [193]:
jane_vtagged = forge.querying.retrieve(jane.id, version="v1")

In [194]:
jane_vtagged._store_metadata.version == 1

True

**Retrieval - Error**

In [195]:
forge.querying.retrieve(jane.id, version=99)

<action> retrieve
<error> resource not found
<args> (<kgforge.specializations.stores.demo_store.DemoStore object at 0x110c47fd0>, 8, 99)


**Searching**

In [196]:
jane = Resource(type="Person", name="Jane Doe")

In [197]:
john = Resource(type="Person", name="John Smith")

In [198]:
associations = Resources(Resource(type="Association", agent=jane),
                         Resource(type="Association", agent=john))

In [199]:
forge.storing.register(associations)

<count> 2
<action> _register
<succeeded> True


In [200]:
p = forge.modeling.paths("Association")

NB: DemoStore is not automatically including templates for nested typed resources yet


You have autocompletion on `p`!

In [201]:
results = forge.querying.search(p.type == "Association")

In [202]:
len(results)

4

In [203]:
type(results[0].agent)

kgforge.core.resources.Resource

In [204]:
# TODO Implement as_dataframe().
# forge.transforming.as_dataframe(results, store_metadata=True)

You have autocompletion on `p`! But also on `p.agent`!

In [205]:
results_filtered = forge.querying.search(p.type == "Association", p.agent.name == "John Smith")

In [206]:
len(results_filtered)

2

In [207]:
print(results_filtered[0])

{
    type: Association
    id: 5
    agent:
    {
        type: Person
        name: John Smith
    }
}


**SPARQL**

In [208]:
# FIXME Add a SPARQL part to DemoStore.
forge.querying.sparql("""
    SELECT ?x, ?name
    WHERE
    {
        ?x type "Association" .
        ?x agent ?name
    }
""")

DemoStore is not supporting sparql()


**Downloading**

In [209]:
# FIXME Implement download() for DemoStore.

In [210]:
# jane = Resource(type="Person", name="Jane Doe")

In [211]:
# association = Resource(type="Association", agent=jane, distribution=)

In [212]:
# forge.querying.download(association, "distribution.contentUrl", "./downloaded/")